In [1]:
from __future__ import print_function

from miscc.utils import mkdir_p
from miscc.utils import build_super_images
from miscc.losses import sent_loss, words_loss
from miscc.config import cfg, cfg_from_file
from DAMSM_train import train, evaluate, build_models, parse_args
from datasets import TextDataset
from datasets import prepare_data

from model import RNN_ENCODER, CNN_ENCODER

import os
import sys
# 파일 경로 조작 및 시스템 경로 설정
import time
# 코드 실행 시간을 측정
import random
# 난수 생성
import pprint
# 설정 파일을 예쁘게 출력하는 용도
import datetime
import dateutil.tz
# 현재 시간과 타임스탬프를 관리
import argparse
import numpy as np
# 배열 연산을 위해 사용
from PIL import Image
# 이미지 저장 및 변환

import torch
import torch.nn as nn
# 기본적인 PyTorch 기능과 신경망 모듈
import torch.optim as optim
# 옵티마이저 관련 모듈
from torch.autograd import Variable
# 자동 미분 기능을 위한 래퍼 (PyTorch 최신 버전에서는 필요 X)
import torch.backends.cudnn as cudnn
# GPU 연산 최적화
import torchvision.transforms as transforms
# 이미지 전처리(transform) 관련 기능 제공

In [2]:
dir_path = os.path.abspath(os.path.join(os.getcwd(), './.'))
# .../image_processing/code
sys.path.append(dir_path)

print(dir_path)

c:\Users\blues\바탕 화면\혜원\개인공부\SIG\2024-SIG\2024_winter_sig\image_processing\code


In [3]:
sys.argv = ['pretrain_DAMSM.py', '--cfg', 'cfg/DAMSM/coco.yml', '--gpu', '1']
args = parse_args()
print(args)

Namespace(cfg_file='cfg/DAMSM/coco.yml', gpu_id=1, data_dir='', manualSeed=None)


In [8]:
if args.cfg_file is not None:
     cfg_from_file(args.cfg_file)

if args.gpu_id == -1:
    cfg.CUDA = False
else:
    cfg.GPU_ID = args.gpu_id

if args.data_dir != '':
    cfg.DATA_DIR = args.data_dir
print('Using config:')
pprint.pprint(cfg)


Using config:
{'B_VALIDATION': False,
 'CONFIG_NAME': 'DAMSM',
 'CUDA': True,
 'DATASET_NAME': 'coco',
 'DATA_DIR': '../data/coco',
 'GAN': {'B_ATTENTION': True,
         'B_DCGAN': False,
         'CONDITION_DIM': 100,
         'DF_DIM': 64,
         'GF_DIM': 128,
         'R_NUM': 2,
         'Z_DIM': 100},
 'GPU_ID': 1,
 'RNN_TYPE': 'LSTM',
 'TEXT': {'CAPTIONS_PER_IMAGE': 5, 'EMBEDDING_DIM': 256, 'WORDS_NUM': 15},
 'TRAIN': {'BATCH_SIZE': 48,
           'B_NET_D': True,
           'DISCRIMINATOR_LR': 0.0002,
           'ENCODER_LR': 0.002,
           'FLAG': True,
           'GENERATOR_LR': 0.0002,
           'MAX_EPOCH': 600,
           'NET_E': '',
           'NET_G': '',
           'RNN_GRAD_CLIP': 0.25,
           'SMOOTH': {'GAMMA1': 4.0,
                      'GAMMA2': 5.0,
                      'GAMMA3': 10.0,
                      'LAMBDA': 1.0},
           'SNAPSHOT_INTERVAL': 5},
 'TREE': {'BASE_SIZE': 299, 'BRANCH_NUM': 1},
 'WORKERS': 1}


In [10]:

print('manualSeed:', args.manualSeed)  # manualSeed 값 출력
if not cfg.TRAIN.FLAG:
    args.manualSeed = 100
elif args.manualSeed is None:
    args.manualSeed = random.randint(1, 10000)
    
print('Updated manualSeed:', args.manualSeed)
random.seed(args.manualSeed)
np.random.seed(args.manualSeed)
torch.manual_seed(args.manualSeed)
if cfg.CUDA:
    torch.cuda.manual_seed_all(args.manualSeed)

manualSeed: 3340
Updated manualSeed: 3340


In [ ]:
now = datetime.datetime.now(dateutil.tz.tzlocal())
timestamp = now.strftime('%Y_%m_%d_%H_%M_%S')
# 현재 시간과 시스템의 로컬 타임존 정보를 가져옴

output_dir = '../output/%s_%s_%s' % \
    (cfg.DATASET_NAME, cfg.CONFIG_NAME, timestamp)
# 디렉토리 경로 생성

model_dir = os.path.join(output_dir, 'Model')
image_dir = os.path.join(output_dir, 'Image')
# 모델과 이미지 저장용 디렉토리 경로 설정
mkdir_p(model_dir)
# mkdir_p : 지정된 디렉토리가 존재하지 않으면 생성하는 함수수
mkdir_p(image_dir)

torch.cuda.set_device(cfg.GPU_ID)
# 저장된 GPU ID로 GPU를 설정한다.
cudnn.benchmark = True
# cuDNN의 최적화 활성화화

In [ ]:
imsize = cfg.TREE.BASE_SIZE * (2 ** (cfg.TREE.BRANCH_NUM-1))
# 입력 이미지의 크기를 결정
batch_size = cfg.TRAIN.BATCH_SIZE

## 이미지 전처리
image_transform = transforms.Compose([
    transforms.Scale(int(imsize * 76 / 64)),
    transforms.RandomCrop(imsize),
    transforms.RandomHorizontalFlip()])

#### train dataset Load
dataset = TextDataset(cfg.DATA_DIR, 'train',
                          base_size=cfg.TREE.BASE_SIZE,
                          transform=image_transform)
print(dataset.n_words, dataset.embeddings_num)
"""
dataset.nwords : 데이터 셋의 단어 수
dataset.embeddings_num : 임베딩 크기 
"""

assert dataset, "Invalid Dataset"
# PyThon에서 단순한 존재 검증 즉, True로 평가되는 값인지 확인
# None 값, 빈 리스트, 빈 문자열, 0, False 같은 값이면 AssertionError 발생

# DataLoader : PyTorch에서 데이터를 배치 단위로 처리하기 위한 도구구
dataloader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))
# drop_last=True : 데이터셋의 크기가 배치 크기로 나누어떨어지지 않는 경우 마지막 배치를 버린다.

#### validation dataset Load
dataset_val = TextDataset(cfg.DATA_DIR, 'test',
                        base_size=cfg.TREE.BASE_SIZE,
                        transform=image_transform)
print(dataset.n_words, dataset.embeddings_num)

dataloader_val = torch.utils.data.DataLoader(
    dataset_val, batch_size=batch_size, drop_last=True,
    shuffle=True, num_workers=int(cfg.WORKERS))


In [ ]:
text_encoder, image_encoder, labels, start_epoch = build_models()
para = list(text_encoder.parameters())
for v in image_encoder.parameters():
    if v.requires_grad:
        para.append(v)
# optimizer = optim.Adam(para, lr=cfg.TRAIN.ENCODER_LR, betas=(0.5, 0.999))
# At any point you can hit Ctrl + C to break out of training early.
try:
    lr = cfg.TRAIN.ENCODER_LR
    for epoch in range(start_epoch, cfg.TRAIN.MAX_EPOCH):
        optimizer = optim.Adam(para, lr=lr, betas=(0.5, 0.999))
        epoch_start_time = time.time()
        count = train(dataloader, image_encoder, text_encoder,
                    batch_size, labels, optimizer, epoch,
                    dataset.ixtoword, image_dir)
        print('-' * 89)
        if len(dataloader_val) > 0:
            s_loss, w_loss = evaluate(dataloader_val, image_encoder,
                                      text_encoder, batch_size)
            print('| end epoch {:3d} | valid loss '
                  '{:5.2f} {:5.2f} | lr {:.5f}|'
                  .format(epoch, s_loss, w_loss, lr))
        print('-' * 89)
        if lr > cfg.TRAIN.ENCODER_LR/10.:
            lr *= 0.98

        if (epoch % cfg.TRAIN.SNAPSHOT_INTERVAL == 0 or
            epoch == cfg.TRAIN.MAX_EPOCH):
            torch.save(image_encoder.state_dict(),
                       '%s/image_encoder%d.pth' % (model_dir, epoch))
            torch.save(text_encoder.state_dict(),
                       '%s/text_encoder%d.pth' % (model_dir, epoch))
            print('Save G/Ds models.')
except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')